# Cantilever beam with 4C

In this example, we will be creating

We will first create the mesh

In [ ]:
# We have to import the required classes and functions from BeamMe
from beamme.core.boundary_condition import BoundaryCondition
from beamme.core.conf import bme
from beamme.core.function import Function
from beamme.core.mesh import Mesh
from beamme.four_c.element_beam import Beam3rLine2Line2
from beamme.four_c.material import MaterialReissner
from beamme.mesh_creation_functions.beam_line import create_beam_mesh_line
from utils.lecture_utils import run_four_c

# In the beginning, we create the mesh container which will hold all elements,
# materials, and boundary conditions.
mesh = Mesh()

# The Load will be linearly increased from 0 to 1 over 1 pseudo-time unit.
function_load = Function([{"SYMBOLIC_FUNCTION_OF_TIME": "t"}])
mesh.add(function_load)

# Here we define the material properties for the beam elements.
# We use a Reissner beam material with specified radius, Young's modulus,
material = MaterialReissner(radius=0.01, youngs_modulus=2.1e11)

# Next, we create a straight beam connecting two points in space.
# For spatial discretization, we use 3 linear two-noded beam elements.
beam_set = create_beam_mesh_line(
    mesh, Beam3rLine2Line2, material, [0, 0, 0], [0.3, 0, 0], n_el=100
)

# To fix the cantilver beam, we apply Dirichlet boundary conditions to all
# positions and rotations at one node.
mesh.add(
    BoundaryCondition(
        beam_set["start"],
        {
            "NUMDOF": 6,
            "ONOFF": [1, 1, 1, 1, 1, 1],
            "VAL": [0, 0, 0, 0, 0, 0],
            "FUNCT": [0, 0, 0, 0, 0, 0],
        },
        bc_type=bme.bc.dirichlet,
    )
)

# The concentrated load at the end an be applied as a Neuman boundary
# condition in z-direction.
mesh.add(
    BoundaryCondition(
        beam_set["end"],
        {
            "NUMDOF": 6,
            "ONOFF": [0, 0, 1, 0, 0, 0],
            "VAL": [0, 0, 50000, 0, 0, 0],
            "FUNCT": [0, 0, function_load, 0, 0, 0],
        },
        bc_type=bme.bc.neumann,
    )
)

# Now we can run the simulation.
run_four_c(
    mesh=mesh,
    simulation_name="cantilever",
    total_time=1.0,
    n_steps=4,
    tol_residuum=1e-5,
)

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets import Button, HBox, Label, Output, VBox

simulation_name = "cantilever"
import pyvista as pv
from IPython.display import display
######
######
######

In [ ]:
def plot_beam_2d(simulation_name):
    """Plot the beam deformation in 2D (XZ-plane) over time steps."""

    # Load the result file
    simulation_path = Path.cwd() / simulation_name
    output_file = simulation_path / f"{simulation_name}-structure-beams.pvd"
    reader = pv.get_reader(output_file)

    # Load the data for all time steps
    steps = reader.time_values
    grid_points = []
    cross_section_resultants_map = {
        "material_axial_force_GPs": "normal",
        "material_bending_moment_2_GPs": "bending",
        "material_shear_force_3_GPs": "shear",
    }
    cell_data = {name: [] for name in cross_section_resultants_map.values()}
    for i_step, time in enumerate(steps):
        reader.set_active_time_point(i_step)
        mesh = reader.read()[0]
        grid_points.append(mesh.points.copy())
        for name in mesh.cell_data.keys():
            if name in cross_section_resultants_map:
                cell_data[cross_section_resultants_map[name]].append(
                    mesh.cell_data[name].copy()
                )

    grid_points = np.array(grid_points)
    for key in cell_data:
        cell_data[key] = np.array(cell_data[key])

    # Get the bounds for the plot
    upper_bound = np.max(grid_points, axis=(0, 1))
    lower_bound = np.min(grid_points, axis=(0, 1))
    dimension = np.linalg.norm(upper_bound - lower_bound)

    # TODO: check for out of plane deformations

    #

    # -------------------------------------------------------
    # Output widget for the plot
    # -------------------------------------------------------
    out = Output()
    state = {"step": 0}

    def plot_step():
        step = state["step"]
        with out:
            out.clear_output(wait=True)
            fig = plt.figure(figsize=(10, 6))
            gs = fig.add_gridspec(
                3,
                2,
                width_ratios=[2, 1],  # <-- adjust these numbers
                # hspace=0.35,            # spacing between rows
                # wspace=0.25             # spacing between columns
            )

            # First plot the beam deformation in XZ-plane
            ax0 = fig.add_subplot(gs[:, 0])
            ax0.plot(grid_points[state["step"], :, 0], grid_points[state["step"], :, 2])
            ax0.set_title(f"Deformed beam")
            ax0.set_xlim(
                lower_bound[0] - dimension * 0.1, upper_bound[0] + dimension * 0.1
            )
            ax0.set_ylim(
                lower_bound[2] - dimension * 0.1, upper_bound[2] + dimension * 0.1
            )
            ax0.grid(True)

            # Plot the cross-section resultants
            ax = [fig.add_subplot(gs[i, 1]) for i in range(3)]
            for i, (name, data) in enumerate(cell_data.items()):
                ax[i].plot(data[state["step"]])
                ax[i].set_title(f"{name.capitalize()} along beam")
                ax[i].grid(True)

            # plt.figure(figsize=(6,4))
            # # plt.plot(grid_points[state["step"],:,0], grid_points[state["step"],:,2])
            # plt.plot(cell_data["bending"][state["step"]])
            # # plt.xlim(lower_bound[0] - dimension*0.1, upper_bound[0] + dimension*0.1)
            # # plt.ylim(lower_bound[2] - dimension*0.1, upper_bound[2] + dimension*0.1)
            # plt.title(f"Step = {state['step']}, Time = {steps[state['step']]:.3f}")
            # plt.grid(True)
            # # plt.gca().set_aspect('equal', 'box')

            plt.tight_layout()
            plt.show()

    # -------------------------------------------------------
    # Step label
    # -------------------------------------------------------
    step_label = Label(f"Step: {state['step']}")

    def update_step_label():
        step_label.value = f"Step: {state['step']}"

    # -------------------------------------------------------
    # Buttons + logic
    # -------------------------------------------------------
    btn_first = Button(description="⏮ First")
    btn_prev = Button(description="◀ Prev")
    btn_next = Button(description="Next ▶")
    btn_last = Button(description="Last ⏭")

    def go_first(b):
        state["step"] = 0
        update_step_label()
        plot_step()

    def go_prev(b):
        if state["step"] > 0:
            state["step"] -= 1
        update_step_label()
        plot_step()

    def go_next(b):
        if state["step"] < len(steps) - 1:
            state["step"] += 1
        update_step_label()
        plot_step()

    def go_last(b):
        state["step"] = len(steps) - 1
        update_step_label()
        plot_step()

    btn_first.on_click(go_first)
    btn_prev.on_click(go_prev)
    btn_next.on_click(go_next)
    btn_last.on_click(go_last)

    # -------------------------------------------------------
    # Layout (buttons row, label row, then plot)
    # -------------------------------------------------------
    controls = HBox([btn_first, btn_prev, btn_next, btn_last])
    labels = HBox([step_label])

    ui = VBox([controls, labels, out])

    # Show initial plot
    plot_step()
    display(ui)


plot_beam_2d("cantilever")